In [1]:
import os
import pandas as pd
import glob
import openpyxl
# from openpyxl import load_workbook
from openpyxl.styles import Font, NamedStyle, Alignment
from openpyxl.utils.dataframe import dataframe_to_rows

In [2]:
def get_excel_files(user_name: str) -> list:
    all_files = os.listdir(os.path.join("C:\\Users","Marcos","Desktop", "Meses"))
    excel_files = []
    for file in all_files:
        if file.endswith(".xlsx"):
            excel_files.append("C:/Users/Marcos/Desktop/Meses/" + file)
    return excel_files

In [3]:
cuits = pd.read_excel('C:/Users/Marcos/Desktop/CUITS.xlsx')
file_paths = get_excel_files(1)

In [8]:
i=0
for c in range(len(cuits)):
    if i != len(file_paths):
        alldata_emitidos = pd.DataFrame()
        alldata_recibidos = pd.DataFrame()
        for file_path in file_paths:
            if str(cuits.iloc[c,1]) in file_path:
                if 'Emitidos' in file_path:
                    alldata_emitidos = pd.concat([alldata_emitidos, pd.read_excel(file_path,skiprows=1)], axis=0, ignore_index=True).reset_index(drop=True)
                else:
                    alldata_recibidos = pd.concat([alldata_recibidos, pd.read_excel(file_path,skiprows=1)], axis=0, ignore_index=True).reset_index(drop=True)  
                i+=1
        if len(alldata_emitidos) > 0:
            alldata_emitidos['Fecha'] = pd.to_datetime(alldata_emitidos['Fecha'], format='%d/%m/%Y')
            alldata_emitidos = alldata_emitidos.sort_values(by='Fecha', ascending=True).reset_index(drop=True)
            alldata_emitidos['Fecha'] = alldata_emitidos['Fecha'].dt.strftime('%d/%m/%Y')
            data_nueva = alldata_emitidos[['Fecha', 'Tipo', 'Número Desde', 'Denominación Receptor', 'Imp. Total']]
            excel_filepath = f'C:/Users/Marcos/Desktop/Oficina/Monotributo/{cuits.iloc[c, 0]}.xlsx'
            data_excel = pd.read_excel(excel_filepath, sheet_name="VENTAS NUEVO", usecols="A:E", skiprows=5)
            size = len(data_excel)
            wb = openpyxl.load_workbook(excel_filepath)
            sheet = wb['VENTAS NUEVO']
            for row_idx, row in enumerate(openpyxl.utils.dataframe.dataframe_to_rows(data_nueva, index=False, header=False), (size+7)):
                for col_idx, value in enumerate(row, 1):
                    sheet.cell(row=row_idx, column=col_idx, value=value)
            for row in wb["VENTAS NUEVO"].iter_rows(min_row=(size+7), max_row=(size+len(data_nueva)+7), min_col=1, max_col=5):
                for cell in row:
                    cell.font = Font(name="Calibri", size=11) 
                    if cell.column == 1:
                        cell.alignment = Alignment(horizontal="right")  
                    elif cell.column == 3:
                        cell.alignment = Alignment(horizontal="center")  
            wb.save(excel_filepath)
        if len(alldata_recibidos) > 0:
            alldata_recibidos['Fecha'] = pd.to_datetime(alldata_recibidos['Fecha'], format='%d/%m/%Y')
            alldata_recibidos = alldata_recibidos.sort_values(by='Fecha', ascending=True).reset_index(drop=True)
            alldata_recibidos['Fecha'] = alldata_recibidos['Fecha'].dt.strftime('%d/%m/%Y')
            data_nueva = alldata_recibidos[['Fecha', 'Tipo', 'Número Desde', 'Denominación Emisor', 'Imp. Total']]
            excel_filepath = f'C:/Users/Marcos/Desktop/Oficina/Monotributo/{cuits.iloc[c, 0]}.xlsx'
            data_excel = pd.read_excel(excel_filepath, sheet_name="COMPRAS NUEVO", usecols="A:E", skiprows=5)
            size = len(data_excel)
            wb = openpyxl.load_workbook(excel_filepath)
            sheet = wb['COMPRAS NUEVO']
            for row_idx, row in enumerate(openpyxl.utils.dataframe.dataframe_to_rows(data_nueva, index=False, header=False), (size+7)):
                for col_idx, value in enumerate(row, 1):
                    sheet.cell(row=row_idx, column=col_idx, value=value)
            for row in wb["COMPRAS NUEVO"].iter_rows(min_row=(size+7), max_row=(size+len(data_nueva)+7), min_col=1, max_col=5):
                for cell in row:
                    cell.font = Font(name="Calibri", size=11) 
                    if cell.column == 1:
                        cell.alignment = Alignment(horizontal="right")  
            wb.save(excel_filepath)
    else:
        break
